In [1]:
from pathlib import Path
from topagnps2cche1d import Watershed

In [5]:
# root_dir = Path("../data_inputs/goodwin_creek_1m/")
root_dir = Path(
    "../data_inputs/goodwin_creek_1m_1_and_3_reaches/TopAGNPS_DataSets/3_Reach"
)
# root_dir = Path("../data_inputs/topagnps_ohio_files/")

path_to_reaches = root_dir / "AnnAGNPS_Reach_Data_Section.csv"
path_to_cells = root_dir / "AnnAGNPS_Cell_Data_Section.csv"
path_to_reaches_raster = root_dir / "AnnAGNPS_Reach_IDs.asc"
path_to_agflow_reaches = root_dir / "AgFlow_Reach_Data.csv"

In [6]:
watershed = Watershed()
watershed.import_topagnps_reaches_network(path_to_reaches)
watershed.read_reaches_geometry_from_topagnps_asc_file(path_to_reaches_raster)
watershed.assign_strahler_number_to_reaches()
watershed.determine_reaches_us_ds_direction()
watershed.import_topagnps_cells(path_to_cells)
# watershed.ignore_reaches_with_strahler_leq(1)

Reading Reaches: 100%|██████████| 3/3 [00:00<00:00, 83.18it/s]


In [4]:
watershed.cells[23].receiving_reach_id

2

In [14]:
import networkx as nx

graph = watershed.current_graph

for node in graph.nodes:
    print(node, graph.in_degree(node))

3 0
2 2
4 0


In [5]:
reaches = watershed.reaches
for k, v in reaches.items():
    if not v.ignore:
        print(v)
    # break

-----------------------------
Reach            : 2
Receiving Reach  : 1
Slope            : 1e-05
Strahler Number  : 3
Ignore?          : False
(US/DS) Nodes    : (2, 1)
-----------------------------
Reach            : 3
Receiving Reach  : 2
Slope            : 0.00097
Strahler Number  : 3
Ignore?          : False
(US/DS) Nodes    : (77, 3)
-----------------------------
Reach            : 4
Receiving Reach  : 3
Slope            : 0.00251
Strahler Number  : 2
Ignore?          : False
(US/DS) Nodes    : (78, 162)
-----------------------------
Reach            : 5
Receiving Reach  : 4
Slope            : 1e-05
Strahler Number  : 1
Ignore?          : False
(US/DS) Nodes    : (163, 191)
-----------------------------
Reach            : 6
Receiving Reach  : 4
Slope            : 1e-05
Strahler Number  : 1
Ignore?          : False
(US/DS) Nodes    : (192, 237)
-----------------------------
Reach            : 7
Receiving Reach  : 3
Slope            : 0.00357
Strahler Number  : 3
Ignore?          : 

In [7]:
reach2 = watershed.reaches[2]
reach3 = watershed.reaches[3]
reach4 = watershed.reaches[4]

In [25]:
from topagnps2cche1d.cell import Cell

In [32]:
cell = Cell(id=12)
cell.type

'right'

In [18]:
print(reach3.nodes[reach3.us_nd_id])

-----------------------------
Node            : 31
TYPE            : None
USID            : None
DSID            : 32
US2ID           : None
COMPUTEID       : None
(x,y)           : (240228.114013, 3795320.620724)
(row,col)       : (179, 228)


In [19]:
reach3 = watershed.reaches[3]
print(reach3)

-----------------------------
Reach            : 3
Receiving Reach  : 2
Slope            : 0.01082
Strahler Number  : 1
Ignore?          : False
(US/DS) Nodes    : (31, 105)


In [20]:
for node_id, node in reach3.nodes.items():
    print(node)

-----------------------------
Node            : 31
TYPE            : None
USID            : None
DSID            : 32
US2ID           : None
COMPUTEID       : None
(x,y)           : (240228.114013, 3795320.620724)
(row,col)       : (179, 228)
-----------------------------
Node            : 32
TYPE            : None
USID            : 31
DSID            : 33
US2ID           : None
COMPUTEID       : None
(x,y)           : (240227.114013, 3795319.620724)
(row,col)       : (180, 227)
-----------------------------
Node            : 33
TYPE            : None
USID            : 32
DSID            : 34
US2ID           : None
COMPUTEID       : None
(x,y)           : (240226.114013, 3795318.620724)
(row,col)       : (181, 226)
-----------------------------
Node            : 34
TYPE            : None
USID            : 33
DSID            : 35
US2ID           : None
COMPUTEID       : None
(x,y)           : (240224.114013, 3795316.620724)
(row,col)       : (183, 224)
-----------------------------
Node

In [21]:
from topagnps2cche1d.tools import rowcol2latlon_esri_asc, read_esri_asc_file

_, geoMatrix, ncols, nrows, _, _ = read_esri_asc_file(path_to_reaches_raster)
print(ncols, nrows)
print(geoMatrix)
y, x = rowcol2latlon_esri_asc(geoMatrix, 179, 228)
print(x, y)

1001 500
(239999.614013, 1.0, 0.0, 3795500.120724, 0.0, -1.0)
240228.114013 3795320.620724
